In [46]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
# Import all of them 
sales=pd.read_csv("../input/sales_train.csv")

# settings
import warnings
warnings.filterwarnings("ignore")

item_cat=pd.read_csv("../input/item_categories.csv")
item=pd.read_csv("../input/items.csv")
sub=pd.read_csv("../input/sample_submission.csv")
shops=pd.read_csv("../input/shops.csv")
test=pd.read_csv("../input/test.csv")
shops.describe().T

In [47]:
Xtrain.head()

In [48]:
test.describe().T

# price * sales = turnover volume


In [49]:
sales['vol']=sales['item_price']*sales['item_cnt_day']
sales.describe().T

# add category

In [50]:
Train=pd.merge(sales, item, how='inner', on='item_id')

In [51]:
Train.describe().T

# pivot table to create product sales stats per month

In [52]:
Xtrain=pd.pivot_table(sales, values='item_cnt_day', index=['shop_id', 'item_id'],columns=['date_block_num'], aggfunc=np.sum,fill_value=0)

# declining number of product sales

-50%

# seasonal 12mont sales


In [53]:
# library & dataset
import seaborn as sns
df = Xtrain.sum()
print(list(df))

# use the function regplot to make a scatterplot
sns.regplot(x=np.array(list(df.index)), y=np.array(list(df)), fit_reg=True)

verkoop=Xtrain.sum()

for i in range(len(verkoop)-11):
    average = sum(verkoop[i:i+12]) / 12
    decline = sum(verkoop[i:i+12]) / (12.0*130227.0)*100 - 100.0 
    print(i,"-",i+12,"%.0f" % average,"%.2f" % decline,'%')
    
print( 'decline 32%/22 * 34 ', 32/22*34)

# Correlogram top ten categories - sales volume

the top ten categories, and their relation in function of the monthly volumes is increasing, so they are drinving their sales volume with the top categories

cat_month.sort_values(0)[-10:].loc[:,:12]

In [54]:
cat_month=pd.pivot_table(Train, values='vol', index=['item_category_id'],columns=['date_block_num'], aggfunc=np.sum,fill_value=0)

for xi in range(12):
    if xi+24<33:
        cat_month[xi]=( cat_month[xi]+cat_month[xi+12]+cat_month[xi+24] )/3
    else:
        cat_month[xi]=( cat_month[xi]+cat_month[xi+12] )/2
        
import matplotlib.pyplot as plt
import seaborn as sns
 
# with regression
sns.pairplot(cat_month.sort_values(0)[-10:].loc[:,:12], kind="reg")
plt.show()


# pivot turnover per product

In [55]:
Ytrain=pd.pivot_table(sales, values='vol', index=['shop_id', 'item_id'],columns=['date_block_num'], aggfunc=np.sum,fill_value=0)

# Constant Sales Volume

there was an 12% increase and then the sales volume decreased again

In [56]:
# library & dataset
import seaborn as sns
df = Ytrain.sum()
print(list(df))

# use the function regplot to make a scatterplot
sns.regplot(x=np.array(list(df.index)), y=np.array(list(df)), fit_reg=True)
plt.show()

verkoop=Ytrain.sum()

for i in range(len(verkoop)-11):
    average = sum(verkoop[i:i+12]) / 12
    decline = sum(verkoop[i:i+12]) / (12.0*101460394)*100 - 100.0 
    print(i,"-",i+12,"%.0f" % average,"%.2f" % decline,'%')
    


# lets do some ARIMA forecasting

* 12month seasonal effect,thats clear
* best models have AIC 300 and are of this type  ARIMA(1, 0, 0)x(1, 1, 0, 12)12


In [57]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf
lag_acf = acf(verkoop, nlags=120)
lag_pacf = pacf(verkoop, nlags=12, method='ols')
#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(verkoop)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(verkoop)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(verkoop)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(verkoop)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [58]:
from statsmodels.tsa.arima_model import ARIMA
import warnings
import itertools
import statsmodels.api as sm
vrkp=verkoop.diff().fillna(0).values

# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))
warnings.filterwarnings("ignore")
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(vrkp,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue
            

mod = sm.tsa.statespace.SARIMAX(vrkp,
                                order=(1, 1, 0),
                                seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

model = sm.tsa.statespace.SARIMAX(vrkp, order=(1, 1, 1),seasonal_order=(1, 1, 0, 12),enforce_stationarity=False,enforce_invertibility=False )  
results_AR = model.fit(disp=-1)  
plt.plot(vrkp)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-vrkp)**2))

results.plot_diagnostics(figsize=(10, 8))
plt.show()


In [59]:
#corr verkoop producten, comarketing
mcor=Ytrain.corr()
mcor=np.reshape(mcor,(34,34))
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
X = np.arange(0, 34, 1)
Y = np.arange(0,34,1)
X, Y = np.meshgrid(X, Y)
Z = mcor

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(0, 1)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=23)

plt.show()

# classification script

In [61]:
def Klasseer(Mtrain,Mtest,Mlabel,klas,rank,start):
    #data preparation
    #print(Mtotal)
    #Mtotal=Mtotal.fillna(-1)
    #print(Mtotal)
    #Mtrain=Mtotal[Mtotal[labelveld]!=-1]
    #Mtest=Mtotal[Mtotal[labelveld]==-1]
    #Mtest=Mtest.drop(labelveld,axis=1)
    Mlabel=pd.DataFrame( Mlabel.values,columns=['label'] )  #[:len(Mtrain)]
    labelveld='label'
    print('shapes train',Mtrain.shape,'label',Mlabel.shape,'test',Mtest.shape)

    
    #totalA=Mtrain.append(Mtest)
    totalA=np.concatenate((Mtrain,Mtest), axis=0)
    predictionA=pd.DataFrame(Mlabel,columns=[labelveld])    
    #totalA=totalA.drop(labelveld,axis=1)
    #print(totalA.shape,predictionA.shape)
    #print(prediction)
    #faze 1
    # dimmension reduction
    from scipy.spatial.distance import cosine
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.decomposition import TruncatedSVD
    from sklearn.preprocessing import Normalizer
    from sklearn.pipeline import make_pipeline
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, log_loss
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC, LinearSVC, NuSVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,ExtraTreesClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.linear_model import OrthogonalMatchingPursuit,RANSACRegressor,LogisticRegression,ElasticNetCV,HuberRegressor, Ridge, Lasso,LassoCV,Lars,BayesianRidge,SGDClassifier,LogisticRegressionCV,RidgeClassifier,Perceptron

    import seaborn as sns
    import matplotlib.pyplot as plt
    
    
    for ira in range(rank-start,rank+1):
        print('****20% sample test==',ira)
        #Ulsa = lsa.fit_transform(Mtrain.values/255)  #train version
        #print(total)
        if ira!=0:
            if ira<len(totalA.T):
                print("lsa dimmension reduction")                
                svd = TruncatedSVD(ira)
                normalizer = Normalizer(copy=False)
                lsa = make_pipeline(svd, normalizer)
                UlsA = lsa.fit_transform(totalA) #total version
                explained_variance = svd.explained_variance_ratio_.sum()
                print("Explained variance of the SVD step knowledge transfer: {}%".format(
                    int(explained_variance * 100)))    

            else:
                print("no reduction")
                UlsA=totalA
        else:
            print("3D-SVD dimmension reduction")
            u,s,vh=np.linalg.svd(totalA)
            print(u.shape, s.shape, vh.shape)
            UlsA=np.reshape(u, (len(totalA),28*28))            
        #    UlsA = totalA
        #    print("no LSA reduction")
        print('ulsa',UlsA.shape)


        #faze2
        #training model

        #sample
        samlen=int(len(Mlabel)/1)
        X_train, X_test, y_train, y_test = train_test_split(UlsA[:samlen], Mlabel[:samlen],stratify=Mlabel[:samlen], test_size=0.5)
        print("test on 20% sample")
        
        if klas=='Logi':
            classifiers = [
    #    SVC(kernel="rbf", C=0.025, probability=True),  20%
    #    NuSVC(probability=True),
                LogisticRegression(),
                 ]
        if klas=='Quad':
            classifiers = [
                QuadraticDiscriminantAnalysis(),
                 ]           
        if klas=='Rand':
            classifiers = [
                RandomForestClassifier(84),
                 ]               
        if klas=='Extr':
            classifiers = [
                ExtraTreesClassifier(verbose=1,n_jobs=3),
                 ]             
        if klas=='Adab':
            classifiers = [
                AdaBoostClassifier(),
                 ]            
        if klas=='Deci':
            classifiers = [
                DecisionTreeClassifier(),
                 ]
        if klas=='Grad':
            classifiers = [
                GradientBoostingClassifier(),
                 ]            
        if klas=='KNN':
            classifiers = [
                KNeighborsClassifier(n_jobs=4),  
                 ]            
        if klas=='Line':
            classifiers = [
                LinearDiscriminantAnalysis(), 
                 ]  
        if klas=='Gaus':
            classifiers = [
                GaussianNB(),
                 ] 
        if klas=='Perc':
            classifiers = [
                Perceptron(),
                 ]      
        if klas=='Elas':
            classifiers = [
                ElasticNet(random_state=0),
                 ]                 
    # Logging for Visual Comparison
        log_cols=["Classifier", "Accuracy", "Log Loss"]
        log = pd.DataFrame(columns=log_cols)
    
        for clf in classifiers:
            clf.fit(X_train,y_train)
            name = clf.__class__.__name__
        
            print("="*30)
            print(name)
            
            #print('****Results****')
            train_predictions = clf.predict(X_test)
            acc = accuracy_score(y_test, train_predictions)
            print("Accuracy: {:.4%}".format(acc))
        
            train_predictions = clf.predict_proba(X_test)
            ll = log_loss(y_test, train_predictions)
            print("Log Loss: {}".format(ll))
            
            log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
            log = log.append(log_entry)
    
        print("="*30)

    print('*** train complete set==',UlsA[:len(Mlabel)].shape)
     
    clf.fit(UlsA[:len(Mlabel)],Mlabel)
    #on complete trainset

    #pr2=pd.DataFrame(clf.predict_proba(Ulsa),index=list(range(0,len(Ulsa),1)))

    predictionA=pd.DataFrame(clf.predict(UlsA),columns=['pred'],index=range(0,len(UlsA)))
    predictionA[labelveld]=Mlabel 
    print('predict',predictionA.shape)
    predictionA.fillna(-1)
    predictionA['diff']=0
    predictionA['next']=Mlabel
    #abs(prediction[labelveld]-prediction['pred
    collist=sorted( Mlabel.label.unique() )

    print(collist)
    if klas=='Logi':
        predictionA[collist] = pd.DataFrame(clf.predict_log_proba(UlsA))
    if klas!='Logi':
        print(UlsA.shape)
        temp=pd.DataFrame(clf.predict_proba(UlsA))
        print(temp.shape)
        predictionA[collist]=temp
    
    from sklearn.metrics import classification_report, confusion_matrix
    true_labels=predictionA[labelveld][:len(Mtrain)].values.astype('float32')
    predicted_labels = predictionA['pred'][:len(Mtrain)].values.astype('float32')

    cm = confusion_matrix(true_labels, predicted_labels,labels=collist)
    print(classification_report(true_labels, predicted_labels))
    print("Confusion matrix")
    print(cm)
    
    corr=predictionA.drop(['pred','diff'],axis=1).corr()
    f, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(abs(corr), mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True, ax=ax)
    predictionA=predictionA.fillna('0')
    #print('Prediction',prediction.head())
    pred2=predictionA.drop(['pred',labelveld,'diff','next'],axis=1)
    
    print(predictionA.shape)


    return predictionA #['next']

# problem
classes where frequency is only 1
so find them, and double them

In [21]:
train1=pd.DataFrame([])
for xi in range(3000):
    lengte=len( Xtrain[Xtrain[33]==xi] )
    if lengte==1:
        train1=train1.append(Xtrain[Xtrain[33]==xi])
train1

# merge the Xtest with the pivot Sales data

* one of the problems is that 50% of the numbers don't have sales data, so probably the sales could be 0, maybe there are products where the sales can start... there we need to spillover knowhow from one shop to another

In [62]:
Xtest.reset_index().merge(Xtrain.reset_index(),how='left')

# Using the ARIMA knowhow

* we found that there is a 12month lag, and a month lag
* hence we need to know the 12 month difference, and the month difference
* we add those columns to the training data

In [64]:
Ttrain=Xtrain.append(train1)
#append those unique double 
#Xtrain.diff(periods=12,axis=1).dropna(axis=1)
Atrain=Ttrain.T.append(Ttrain.T.diff(periods=12).dropna() )
Atrain=Atrain.append(Ttrain.T.diff(periods=1).dropna() )
Alabel=pd.DataFrame(Ttrain.index,columns=['prid'])

#Atrain.T




In [ ]:
Klasseer(Atrain.fillna(0).T.values,Atrain.T[400000:].values,Ttrain[33],'Extr',40,0)